In [ ]:
%pip install spacy
%pip install vaderSentiment
%pip install pyarrow
%pip install torch
%pip install nltk
%pip install textblob
%pip install -U textblob-de
%pip install transformers
!{sys.executable} -m spacy download de_core_news_lg
!{sys.executable} -m spacy download de_core_news_md

In [5]:
import os
import re
import pandas as pd
from pyarrow import feather
import numpy as np

#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import sys
sys.path.append("GERVader")
from vaderSentimentGER import SentimentIntensityAnalyzer # -- https://github.com/KarstenAMF/GerVADER
from nltk.tag import pos_tag
import sys
import nltk
#nltk.download("vader_lexicon")

from scipy.special import softmax
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

# GERVader

In [115]:
pd.set_option('display.max_colwidth', None)
post_data = pd.read_feather('reddit_res/posts.ftr')
comment_data = pd.read_feather('reddit_res/comments.ftr')
post_data

,title,text,subreddit,created
0,25€ postgebühr aliexpress sendungen,momentan 25€ gaudigebühr post bestellungen aliexpress voriges ständig 25€ post bezahlen bestellwerten wenigen euro wirtschaftlich pakete teilweise angenommen absolute resourcenverschwendung umweltverschmutzung vertrottelte verpackungsverordung ohnehin vorerst mittlerweile sachen aliexpress zwischenzeit bauteile gerne probierten lust ständig 25€ zahlen vat entrichtet gebühr versandart umgehen,austria,2023-05-22 11:28:50
1,frage nachbarländer einkaufen deutlich günstiger klimaticket erreichbar,inspiriert post deutschen deutschlandticket polen einkaufen fahrt günstiger preußen frag nachbarn deutlich günstiger einkaufen deutschland günstiger österreich passau klimaticket erreichbar tschechien slowakei ungarn slowenien günstiger orte günstiger österreich einkaufen klimaticket gilt italien schweiz teurer wien hinzufahren euro sparen rest 3h spartipps,austria,2023-05-22 09:29:17
3,klimabonus,werte mitbürgerinnen klimabonus 2022 4 kontakt zuständigen stelle vertröstet 1 versucht altes bankkonto überweisen 2021 aktualisiert steuerausgleich aufs konto bekommen korrigiert 23 wochen warten 2 kontakt herr überweisung fehlgeschlagen 2tes überwiesen bekomme sodfxo gutscheine zugeschickt 34 wochen dauern 3 kontakt anfang märz vertröstet welle aussendungen 34wochen warten 4 kontakt 1 woche urlaub spital aufenthalt formular warte antwort bereit idioten zustehende geld schenken,austria,2023-05-22 00:18:07
4,klima service lucky car erfahrungen,gerne klimaservice lucky car nähe wien umgebung aufgemacht lucky car nachfragen leistung zufrieden seids preislich standort bezogene unterschiede würd erfahrungen hören,austria,2023-05-21 14:05:15
5,flughafen wien wasser warm/heiß,wasser wasserhähnen toiletten flughafen wien warm heiß raus heizen wasser 4€ mineralwasser kauft woke umweltbewusst,austria,2023-05-20 17:05:23
6,amal,ahnung klimakatastrophe retten klammern hoffnung wundertechnik klammern irrglauben technik politiker korrupt einzusetzen ignorieren thema x200b ists logisch heutigen system konsumismus zwang immerwährenden wirtschaftswachstums klimaproblem gelöst eautos größte greenwashing generation neuerdings geliebten akws helfen energie überfluss wege verbrauchen super server nutzlose kalkulationen diverse blockchains bitcoin miner super server millionen daten speichern exponentiell anhäufen straßen infrastruktur autos größten verschmutzer idee autos wegkommen wirtschaft hängt bauen lösung eautos beschließen 2035 verbrenner straßen plan stromnetz jemals verkraften benötigte energie stammen deutschland vorreiter erneuerbaren energie grad grad geschafft 55 energie stromsektor erneuerbare ersetzen stromsektor 20 gesamtenergieverbrauchs landes x200b gezwungen vereinbarung ewigem wirtschaftswachstum klimaschutz wohlstand gesellschaft hängt drogensüchtiger heroin gesamte konstrukt auseinanderbrechen wohlstand verlieren armes landarme privileg übers klima sorgen,austria,2023-05-20 11:13:41
7,klimakleben entlassen,festkleben chef entlassen freizeit,austria,2023-05-17 18:46:22
10,ö3-jugendstudie gen heiraten lehrpläne klimapolitik,92 prozent einfamilienhaus eigentumswohnung leben hahahahhahah grünen bricht jugend scheiß freu unermessliche scheinbar junge generation deppat,austria,2023-05-16 07:52:50
11,fpö-wählern leben schwer,ausländer nato impfungen radfahrer mächtigen flüchtende kinder klimaaktivisten binneni veganer usa wissenschaft sozialdemokraten greta thunberg ukraine nichtraucherzonen orf tempolimits bio qualitätsjournalismus frauen ausländer begegnungszonen gürtelpools eu fremdsprachen grünen fleischersatzprodukte bildung registrierkassen justiz grünflächen eliten armin wolf lgbtq wort töchter x200b gefunden twitter meinung,austria,2023-05-15 17:45:48
14,mikl-leitner nennt kooperation fpö reibungslos schließt koalition kickl,niederösterreichs landeshauptfrau bilanz monate regierungsarbeit gezogen worte blauen koalitionspartner gefunden wien – monate regiert landeshauptfrau joha

In [24]:
def perform_sentiment_analysis_vader(data):
    data_temp = data.copy()
    sid = SentimentIntensityAnalyzer()
    data_temp["Negative"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["neg"])
    data_temp["Positive"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["pos"])
    data_temp["Neutral"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["neu"])
    data_temp["Compound"] = data_temp["text"].apply(lambda x:sid.polarity_scores(x)["compound"])
    data_temp["Overall"] = ['Positive' if score >= 0.05 else 'Negative' if score <= -0.05 else 'Neutral' for score in data_temp["Compound"]]
    return data_temp

In [25]:
post_data_vader = perform_sentiment_analysis_vader(post_data)
comment_data_vader = perform_sentiment_analysis_vader(comment_data)

In [26]:
overall_sentiments_vader = pd.concat([post_data_vader["Overall"].value_counts(), comment_data_vader["Overall"].value_counts()], axis=1)
overall_sentiments_vader.columns = ["Post Data Sentiment", "Comment Data Sentiment"]
overall_sentiments_vader

,Post Data Sentiment,Comment Data Sentiment
Positive,38,2929
Negative,6,2174
Neutral,6,2246


# TextblobDE

In [27]:
from textblob_de import TextBlobDE

In [28]:
def perform_sentiment_analysis_blob(data):
    data_temp = data.copy()
    data_temp["Polarity"] = data_temp.apply(lambda x: TextBlobDE(x["text"]).sentiment.polarity, axis=1)
    data_temp["Subjectivity"] = data_temp.apply(lambda x: TextBlobDE(x["text"]).sentiment.subjectivity, axis=1)
    data_temp["Overall"] = ['Negative' if score < 0.0 else 'Neutral' if score == 0.0 else 'Positive' for score in data_temp["Polarity"]]
    return data_temp

In [29]:
post_data_textblob = perform_sentiment_analysis_blob(post_data)
comment_data_textblob = perform_sentiment_analysis_blob(comment_data)

In [30]:
overall_sentiments_blob = pd.concat([post_data_textblob["Overall"].value_counts(), comment_data_textblob["Overall"].value_counts()], axis=1)
overall_sentiments_blob.columns = ["Post Data Sentiment", "Comment Data Sentiment"]

In [31]:
overall_sentiments_blob

,Post Data Sentiment,Comment Data Sentiment
Positive,19,1773
Neutral,16,3761
Negative,15,1815


# Pre-trained GeRiBERTa model

In [128]:
import spacy
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tqdm as notebook_tqdm


model_name = 'oliverguhr/german-sentiment-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [129]:
def perform_sentiment_analysis(text):
    # Tokenize and truncate the text
    tokens = tokenizer.tokenize(text)
    max_length = tokenizer.model_max_length 
    tokens = tokens[:max_length-2] 
    tokens = ['[CLS]'] + tokens + ['[SEP]']

    # Convert tokens to input IDs, convert them to tensor
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  

    # Feed the tokenized input to the pre-trained model, don't track gradient
    with torch.no_grad():
        logits = model(input_ids)[0]

    # Convert the logit to a probability
    scores = softmax(logits) 
    sentiment_scores = scores.tolist()
    rounded_scores = [[round(score, 4) if isinstance(score, float) else score for score in sublist] for sublist in sentiment_scores]
    
    # Get the predicted sentiment label
    predicted_label = logits.argmax().item()
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[predicted_label]

    return sentiment, rounded_scores

In [130]:
post_data_geriberta = post_data.copy()
comment_data_geriberta = comment_data.copy()

post_data_geriberta[["Sentiment", "Scores"]] = post_data_geriberta["text"].apply(perform_sentiment_analysis).apply(pd.Series)
post_data_geriberta["Sentiment"] = post_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())

comment_data_geriberta[["Sentiment", "Scores"]] = comment_data_geriberta["text"].apply(perform_sentiment_analysis).apply(pd.Series)
comment_data_geriberta["Sentiment"] = comment_data_geriberta["Sentiment"].apply(lambda x: x.capitalize())

Token indices sequence length is longer than the specified maximum sequence length for this model (1039 > 512). Running this sequence through the model will result in indexing errors


In [131]:
overall_sentiments_geriberta = pd.concat([post_data_geriberta["Sentiment"].value_counts(), comment_data_geriberta["Sentiment"].value_counts()], axis=1)
overall_sentiments_geriberta.columns = ["Post Data Sentiment", "Comment Data Sentiment"]

In [132]:
overall_sentiments_geriberta 

,Post Data Sentiment,Comment Data Sentiment
Positive,28,3315
Neutral,18,3275
Negative,4,758


In [ ]:
# https://huggingface.co/oliverguhr/german-sentiment-bert 

# German Sentiment Classification with Bert 

In [ ]:
from germansentiment import SentimentModel

In [123]:
def add_sentiment_scores(df):
    model = SentimentModel()
    sentiment_scores = []
    sentiments = []
    for index, row in df.iterrows():
        text = [row["text"]]
        classes, probabilities = model.predict_sentiment(text, output_probabilities=True)
        rounded_scores = [[round(score, 4) if isinstance(score, float) else score for score in sublist] for sublist in probabilities[0]]
        sentiment_scores.append(rounded_scores)
        sentiments.append(classes[0])
    
    df["Sentiment"] = sentiments
    df["Sentiment score"] = sentiment_scores
    return df

In [124]:
post_data_germansentiment = add_sentiment_scores(post_data.copy())
comment_data_germansentiment = add_sentiment_scores(comment_data.copy())

In [126]:
overall_sentiments_ger = pd.concat([post_data_germansentiment["Sentiment"].value_counts(), comment_data_germansentiment["Sentiment"].value_counts()], axis=1)
overall_sentiments_ger.columns = ["Post Data Sentiment", "Comment Data Sentiment"]

In [127]:
overall_sentiments_ger

,Post Data Sentiment,Comment Data Sentiment
neutral,27,3346
negative,19,3233
positive,4,769


In [49]:
from nltk.tokenize import word_tokenize
from collections import Counter

word_counts = Counter()
for corpus in post_data["text"]:
    words = word_tokenize(corpus)
    word_counts.update(words)

sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
print("Most frequently used sentiment words:")
for word, count in sorted_words:
    print(word, ":", count)

Most frequently used sentiment words:
fr : 47
ber : 18
wrde : 17
wien : 15
linz : 14
studium : 11
russland : 11
sterreich : 10
berhaupt : 10
leute : 10
wre : 10
stadt : 10
klimaticket : 9
knnen : 9
thema : 9
sp : 8
fragen : 8
stellen : 8
graz : 8
gibts : 7
ukraine : 7
fahren : 7
sowieso : 7
radweg : 7
post : 6
gerne : 6
1 : 6
anfang : 6
erfahrungen : 6
x200b : 6
leben : 6
usa : 6
meinung : 6
land : 6
regierung : 6
politik : 6
ned : 6
komplett : 6
gnstiger : 5
deutschland : 5
klimabonus : 5
4 : 5
2 : 5
woche : 5
generation : 5
energie : 5
grad : 5
radfahrer : 5
opposition : 5
gelesen : 5
misstrauensantrag : 5
strae : 5
ziemlich : 5
finde : 5
probleme : 5
egal : 5
krieg : 5
25 : 4
teilweise : 4
deutschen : 4
einkaufen : 4
gilt : 4
kontakt : 4
bekommen : 4
bekomme : 4
3 : 4
super : 4
straen : 4
wrden : 4
klima : 4
natrlich : 4
scheinbar : 4
miklleitner : 4
vp : 4
fp : 4
htte : 4
partner : 4
offenbar : 4
bitte : 4
darber : 4
themen : 4
doof : 4
gruppe : 4
unternehmen : 4
eindruck : 4
auto 

In [45]:
with open("stopwords/stopwords-de.txt", "r") as file:
    additional_stop_words = set(file.read().splitlines())
    
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(post_data_vader['text'])

def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

n_top_words = 10
print_topics(lda, count_vectorizer, n_top_words)


Topic #0:
studium spö wiederverwendbarer regierung graz misstrauensantrag opposition energie günstiger klimaticket

Topic #1:
linz stadt mistkübeln erfahrungen österrechische liegen gackisackerlspender bratislava klimaticket wiederverwendbarer

Topic #2:
russland ukraine komplett usa krieg linz wiederverwendbarer gerne politik land

Topic #3:
radweg radfahrer fahren ned stellen österrechische sowieso stadt deutschland linz

Topic #4:
ausländer eu möglichkeit frage kaufen geographie raus grünen vortrag würds


In [ ]:
     # Don't track gradient.
        with torch.no_grad():
            # Tokens are made up of CLS token, text converted to tokens, and SEP token.
            tokens = ["[CLS]"] + self.tokenizer.tokenize(text) + ["[SEP]"]
            # Convert tokens to input IDs; convert them to tensor, unsqueeze, put it to device.
            input_ids = (
                torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens))
                .unsqueeze(0)
                .to(self.device)
            )
            # Create attention mask from input IDs.
            attention_mask = (input_ids != 0).long()
            # Get logit (log-odds) of sentiment being positive from the model.
            positive_logit = self.model(
                input_ids=input_ids, attention_mask=attention_mask
            )
            # Convert the logit to a probability.
            positive_probability = torch.sigmoid(positive_logit.unsqueeze(-1)).item()
            # Convert the probability to a percentage.
            positive_percentage = positive_probability * 100
            # Conver probability to boolean.
            is_positive = positive_probability > 0.5
            # Return sentiment and percentage.
            if is_positive:
                return "Positive", int(positive_percentage)
            else:
                return "Negative", int(100 - positive_percentage)